# Load Chatbot


In [ ]:
!pip install openai
!pip install datasets
from datasets import load_dataset, load_from_disk
import openai
import google.generativeai as genai

import os
import copy
import ast
from collections import Counter
import re
!pip install datasets
!pip install -q -U google-generativeai


# Mount to google drive
from google.colab import drive
drive.mount('/content/drive')

# Change it to your google drive path where this notebook located.
drive_path = '/content/drive/MyDrive/Projects/CryptoniteAnalysis/'
os.chdir(drive_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not cu

In [ ]:
# @title GPT Chatbot
API_KEY="YOUR OPENAI API KEY"

# define the openai interface
def try_query_GPT(**request_body):
    client = openai.OpenAI(api_key=API_KEY)
    response = client.chat.completions.create(**request_body)
    return response

def accept_gpt_response(response):
    res_stop = True
    # first check if the response is complete
    if not response.choices[0].finish_reason == "stop":
        res_stop = False

    # Other checks in the future
    return res_stop

def query_GPT(**request_body):
    response = try_query_GPT(**request_body)
    # if response failed
    timeout = 0
    while not accept_gpt_response(response):
        response = try_query_GPT(**request_body)
        timeout += 1
        if timeout > 10:
            raise Exception("Query failed")
    return response.choices[0].message.content

default_request_body = {
    "model": "gpt-4o-mini",
    "messages": [{"role": "system", "content": "You are a helpful assistant."}],
    "temperature": 0.7,
}


class GPTChatBot:
    def __init__(self, initial_request_body=default_request_body):
        if "messages" not in initial_request_body:
            raise ValueError("messages not in request_body")
        if "model" not in initial_request_body:
            raise ValueError("model not in request_body")
        self.initial_request_body = copy.deepcopy(initial_request_body)

        self.chat_history = self.initial_request_body["messages"]

    def chat(self, prompt):
        # query ChatGPT, but do not add the conversation to history
        temp_request_body = copy.deepcopy(self.initial_request_body)
        temp_request_body["messages"].append({"role": "user", "content": prompt})
        response = query_GPT(**temp_request_body)
        return response

    def set_chat_history(self, chat_history):
        self.chat_history = chat_history


In [ ]:
# @title Gemini Chatbot
GEMINI_KEY="YOUR GEMINI API KEY"

# define the openai interface
def try_query_Gemini(**request_body):
    model = request_body["model"]
    chat = model.start_chat(
        history=request_body['history']
    )
    prompt = request_body["prompt"]
    response = chat.send_message(prompt, generation_config=request_body["generation_config"])
    return response

def accept_Gemini_response(response):
    res_stop = True
    # first check if the response is complete
    if not response._done:
        res_stop = False

    # Other checks in the future
    return res_stop

def query_Gemini(**request_body):
    response = try_query_Gemini(**request_body)
    # if response failed
    timeout = 0
    while not accept_Gemini_response(response):
        response = try_query_Gemini(**request_body)
        timeout += 1
        if timeout > 10:
            raise Exception("Query failed")
    return response


class GeminiChatBot:
    def __init__(self, system_prompt="You are a helpful assistant.", gemini_model="gemini-1.5-flash", temperature=0.7):
        genai.configure(api_key=GEMINI_KEY)
        self.model = genai.GenerativeModel(model_name=gemini_model, system_instruction=system_prompt)
        self.generation_config = genai.types.GenerationConfig(temperature=temperature)
        self.chat_history = []




    def chat(self, prompt):
        '''
        for gemini we are not puting a interactive chatbot with history, just zero shot.
        No need to add the print feature
        '''
        request_body = {
            "model": self.model,
            "generation_config": self.generation_config,
            "history" : self.chat_history,
            "prompt": prompt,
        }
        response = query_Gemini(**request_body)

        return response.text
    def set_chat_history(self, chat_history):
        self.chat_history = chat_history



# Anagram

In [ ]:
def solve_simple_anagram(sample, chat_bot, information_extractor):
    # get the clue
    clue = sample['clue']
    # Remove the tuple at the end (assuming there are no other parenthsis in the clue.), Remove periods and commas
    clue = re.sub(r"[^a-zA-Z']", ' ', clue)
    # Split the clue into a list of words
    word_list = clue.split()
    # get the word length for each word in clue
    word_len_list = [len(word) for word in word_list]

    # get enumeration
    enumeration = sample["enumeration"]
    # get the number of letters of the answer (add all the numbers in enumeration)
    if '-' in enumeration:
        split_sign = '-'
    else:
        split_sign = ','
    numbers_list = enumeration.strip('()').split(split_sign)
    answer_letter_numbers = [int(num) for num in numbers_list]
    # get the word length for answer
    answer_letter_len = sum(answer_letter_numbers)
    # get number of words (see how many numbers are there in enumeration)
    answer_word_len = len(answer_letter_numbers)

    # get all potential anagram phrases
    potential_anagram_phrases = []
    for k in range(len(word_list)-1):
        continue_checking = False
        for i in range(len(word_list) - k):
            if sum(word_len_list[i:i+k+1]) == answer_letter_len:
                potential_anagram_phrases.append(" ".join(word_list[i:i+k+1]))
            if sum(word_len_list[i:i+k+1]) < answer_letter_len:
                # as long as there is still one combination of k words that is smaller than answer_letter_len, we continue
                continue_checking = True
        if continue_checking == False:
            break

    # if there is no potential anagrams from the words of clue, then it cannot be solved.
    if len(potential_anagram_phrases) == 0:
        return None

    # Iteration 1: Get Shuffle_phrase, Indicator, Definition
    prompt = f"Given the cryptic crossword puzzle '{clue}', I know that it is a anagram type crossword puzzle. The hint number suggests that the shape of the answer is {enumeration}, so that means the phrase we want to shuffle have exacty {answer_letter_len} english letters. Here are all the phrases in the clue that satisfies this requirement: {potential_anagram_phrases}. I need to pick from them the most likely phrase to shuffle. \nFor each of the potential phrase to shuffle, what's left in the clue will consists of the indicator and definition (Indicator are words that indicate this clue is an anagram, definition are words that defines the answer). I want you to solve this problem by the following steps: You will first try to identify the indicator and definition from what's left inside the clue. Then you will look at the potential indicator and definition you picked, and decide if the indicator is actually likely to be an indicator of an anagram. If yes, then the phrase is likely to be the phrase we want to shuffle."
    response = chat_bot.chat(prompt)
    # extract information
    prompt_extract = f"Given the output:\n{response}, \nwhat are all the phrase to shuffle, the indicator and the definition? Give me in the form of tuple of three strings: (<phrase>, <indicator>, <definition>), I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)
    parsed_tuple = ast.literal_eval(response_extract)
    shuffle_phrase, indicator, definition = parsed_tuple

    # Iteration 2: Perform Anagram and Get Answer
    if answer_word_len > 1:
        # Prompt for two words answer (Reminder: add possibility to invert the order.)
        prompt_3 = f"Given the cryptic crossword puzzle {clue}, I know that it is a anagram type crossword puzzle becasue {indicator} is an indicator phrase. The hint number suggests that the shape of the answer is {enumeration}, so that means the phrase we want to shuffle have exacty {answer_letter_len} english letters. We know that the phrase we want to shuffle is {shuffle_phrase}, because this phrase have exactly {answer_letter_len} english letters. Now, given this phrase to shuffle, I want you to follow these steps to find the answer: since the answer have more than one word, you will list all the letters that is the avalible for building words. Then you will try to find the first word, and see what letters are left after building this first word. Finally you will find what word the rest of the letters can form. (try several times if it didn't work) There might be many, if there are many possibilities, you shoud consider which one fits the definition the best. "

    else:
        # prompt for one word answer: shuffle letters to get the results
        prompt_3 = f"Given the cryptic crossword puzzle {clue}, I know that it is a anagram type crossword puzzle becasue {indicator} is an indicator phrase. The hint number suggests that the shape of the answer is {enumeration}, so that means the phrase we want to shuffle have exacty {answer_letter_len} english letters. We know that the phrase we want to shuffle is {shuffle_phrase}, because this phrase have exactly {answer_letter_len} english letters. Now, given this phrase to shuffle, I want you to follow these steps to find the answer: you will first list all the letters that is the avalible for building words, and then base on these letter, you will find the answer for this puzzle. "
    response = chat_bot.chat(prompt_3)
    # extract information
    prompt_extract = f"Given the output:\n{response}, What is the answer? I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)

    # return the answer
    return response_extract

# Charade
Solve simple Charade doesn't need iterative prompting, just Naive CoT + In Context Learning

In [ ]:
def solve_charade(sample,chat_bot, information_extractor):
    clue = sample['clue']
    enumeration = sample['enumeration']
    # get enumeration
    enumeration = sample["enumeration"]
    # get the number of letters of the answer (add all the numbers in enumeration)
    if '-' in enumeration:
        split_sign = '-'
    else:
        split_sign = ','
    numbers_list = enumeration.strip('()').split(split_sign)
    answer_letter_numbers = [int(num) for num in numbers_list]
    # get the word length for answer
    answer_letter_len = sum(answer_letter_numbers)
    # get number of words (see how many numbers are there in enumeration)
    answer_word_len = len(answer_letter_numbers)

    prompt = f"""in cryptic crossword puzzles there is a type called charade. each word or phrase in the clue represents another little piece of the answer, presumably in a misleading way, and you stitch them together to get a result.
    The clue is build with definition and components. by default the components should be used in the order they're presented in, but words like "after" can indicate re-arrangement.
    Charade Clue Structure: The clue contains these parts -
    1. Main Definition
    2. Charade Component Definitions - Definitions of the parts that make up the solution.
    the answer must be logical with the definition.
    the numbers at the end of the clue represent the amount of letter at each word.

    the solution MUST be built from the components and the final answer must suit the definition.
    also they amount of letters must fit to the numbers at the end of the clue.
    here are few examples:

    1:
    clue: "Noodles," mafia man, is coming after you (4)
    answer: UDON
    explanation: The answer is UDON, a type of noodle (noodles is the definition), where DON (a “mafia man”, a component) comes after the letter U (“you”, another component)

    2:
    clue: Small crew’s power source (5)
    answer: STEAM
    explanation: The answer is STEAM, a type of "power source" (this is the definition), and the wordplay is S (short for “small”) plus TEAM (a synonym for “crew”)

    3:

    clue: Wet season soon after Monday (7)
    answer: MONSOON
    explanation: Put SOON after MON, a common abbreviation for “Monday,” to get MONSOON (“wet season”, this is the definition)

    now i want you to solve me the next puzzle which is a charade type. the components MUST be logical and also the answer must be reasonable with the definition.
    Consider multiple interpretations for each component and explore all possible combinations to fit the definition and letter count

    clue: {clue}
    the answer MUST contain {answer_word_len} words {answer_letter_len} letters.
    the answer MUST be derived from the components.
    the answer MUST fit to the definition.

    what is the answer?"""
    response = chat_bot.chat(prompt)
    prompt_extract = f"Given the output:\n{response}, What is the answer? I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)

    return response_extract

# Containers
Solve simple Containers doesn't need iterative prompting, just Naive CoT + In Context Learning

# Letter Selection - Deletion
Solve simple Containers doesn't need iterative prompting, just Naive CoT + In Context Learning

# Double Definition

In [ ]:
def solve_double_definition(sample, chat_bot, information_extractor):
    clue = sample['clue']
    enumeration = sample['enumeration']

    # Get the two definition from the sentence
    prompt = f"Given the cryptic crossword puzzle {clue}, I know that it is a double definition type crossword puzzle, which means the clue may, rather than having a definition part and a wordplay part, have two definition parts. Can you split this sentence, and tell me what are the two definition phrases of the answer? In order to be more accurate, I want you to follow the following steps: \nFIRST you will try to find the first phrase that might be the first definition (start with the first word and check how much of the sentence can form a meaningful definition). \nSecond, you will see what is left in the sentence. \n THIRD, you will determine if your division is correct by checking if the phrase that's left can be a meaningful definiton of something. \nFOURTH, you will look at each definitions, and interpret what they could mean, what could they be referring to (there might be multiple meanings, and I want you to list all possible meanings of this definition). \FIFTH, you will put all the interpretations together into a list. "
    response = chat_bot.chat(prompt)
    # extract information
    prompt_extract = f"Given the output:\n{response}, What is the the two definition phrases, and what are the bucket of interpretations?  Give me in the form of tuple of three strings: (<definition 1>, <definition 2>, <interpretations>) I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)
    definition_1, definition_2, interpretations = ast.literal_eval(response_extract)

    # Get the Answer
    prompt = f"Given the cryptic crossword puzzle {clue}, I know that it is a double definition type crossword puzzle, which means the clue may, rather than having a definition part and a wordplay part, have two definition parts. After looking at the clue, I think the first part of the definition is {definition_1}, and the rest of the sentence formed the second part of the definition: {definition_2}. The hint number suggests that the shape of the answer is {enumeration}. Therefore, the possible answer should be a {enumeration} shaped word that fits both definitions, something that relates to '{interpretations}'.So now, I want you to solve the problem this way:\nFIRST, you will think of some words with shape {enumeration}, that relates to '{interpretations}' ---- words that fits under the definiton of both '{definition_1}' and '{definition_2}'. \n SECOND, for each word you listed, check if they can both fit the definition of '{definition_1}' and '{definition_2}', and give an brief explanation. \nFINALLY, conclude the answer based on your analysis."
    response = chat_bot.chat(prompt)
    # extract information
    prompt_extract = f"Given the output:\n{response}, What is the answer? I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)

    return response_extract


# Hidden Words

In [ ]:
def solve_simple_hidden_word(sample, chat_bot, information_extractor):
    clue = sample["clue"]
    # get enumeration
    enumeration = sample["enumeration"]
    # get the number of letters of the answer (add all the numbers in enumeration)
    if '-' in enumeration:
        split_sign = '-'
    else:
        split_sign = ','
    numbers_list = enumeration.strip('()').split(split_sign)
    answer_letter_numbers = [int(num) for num in numbers_list]
    # get the word length for answer
    answer_letter_len = sum(answer_letter_numbers)
    # get number of words (see how many numbers are there in enumeration)
    answer_word_len = len(answer_letter_numbers)

    # Iteration 1: get hidden_phrase, indicator, definition
    prompt = f"Give the cryptic crossword puzzle '{clue}', I know that this puzzle is a hidden word type of cryptic crossword puzzles, that means the answer is somewhere written within the clue – either as part of a longer word or across more than one word. What is the indicator phrase that indicates this is a hidden word puzzle? And in that case, which phrase might hide the answer? Finally, which phrase is the definition?"
    response = chat_bot.chat(prompt)
    # extract information
    prompt_extract = f"Given the output:\n{response}, what are the phrase that might hide the answer, the indicator and the definition? Give me in the form of tuple of three strings: (<phrase>, <indicator>, <definition>), I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)
    parsed_tuple = ast.literal_eval(response_extract)
    hidden_phrase, indicator, definition = parsed_tuple

    # Hybrid Step: perform sliding window
    all_possible_hidden_words = []
    hidden_phrase = re.sub(r"[^a-zA-Z]", '', hidden_phrase)
    hidden_phrase = hidden_phrase.lower()

    for i in range(len(hidden_phrase) - answer_letter_len + 1):
        phrase = hidden_phrase[i:i+answer_letter_len]

        # given a phrase of crrect length, make it the correct size (size of enumeration)
        reshaped_string = []
        start = 0
        for num in answer_letter_numbers:
            # Extract the substring of length `num`
            part = phrase[start:start + num]
            reshaped_string.append(part)
            # Move the start index forward by `num`
            start += num
        # Join the parts with a space
        possible_hidden_word = ' '.join(reshaped_string)
        all_possible_hidden_words.append(possible_hidden_word)

    # Iteration 2: Get the Answer
    prompt = f"Give the cryptic crossword puzzle '{clue}', I know that this puzzle is a hidden word type of cryptic crossword puzzles, that means the answer is somewhere written within the clue – either as part of a longer word or across more than one word. I know that the word {indicator} indicates that the phrase '{hidden_phrase}' will have the answer. Also, the phrase '{definition}' is the definition of the phrase: I have already gave you the definition, so you should not find another definition in the clue yourself. So here are all the possible strings that is the shape of {enumeration} that comes from the phrase: {all_possible_hidden_words}\nWhich one of them suits the given definition?"
    response = chat_bot.chat(prompt)
    # extract information
    prompt_extract = f"Given the output:\n{response}, What is the answer? I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)
    return response_extract

# Tips: Initials/Finals

In [ ]:
def solve_initials_finals(sample, chat_bot, information_extractor):

    clue = sample["clue"]
    enumeration = sample["enumeration"]
    # get the number of letters of the answer (add all the numbers in enumeration)
    numbers_list = enumeration.strip('()').split(',')
    enumeration_numbers = [int(num) for num in numbers_list]
    number_of_letters = sum(enumeration_numbers)
    # get number of words (see how many numbers are there in enumeration)
    number_of_words_in_answer = enumeration.count(',') + 1

    # store the answer for comparison
    answer = sample['answer']

    if sample['is_initial']:
        prompt_word_1 = 'initial'
    if sample['is_final']:
        prompt_word_1 = 'final'

    # We will get the initials/finals of the entire sentence (sometimes rest of sentence doesn't work)
    word_list = re.sub(r'[^a-zA-Z]', ' ', clue).lower().split()
    if sample['is_initial']:
        # initial means we will take word[:k] where k = 1
        concat_tips = [word[:1] for word in word_list]
        concat_tips = ''.join(concat_tips)
    else:
        # todo: here we assume initial and final, but no reverse (reverse the order of the letters)
        concat_tips = [word[-1:] for word in word_list]
        concat_tips = ''.join(concat_tips)
    # Now we will sliding window to all the possible answers.
    all_possible_answers = []
    for i in range(len(concat_tips) - number_of_letters + 1):
        phrase = concat_tips[i:i+number_of_letters]

        # given a phrase of crrect length, make it the correct size (size of enumeration)
        reshaped_string = []
        start = 0
        for num in enumeration_numbers:
            # Extract the substring of length `num`
            word = phrase[start:start + num]
            reshaped_string.append(word)
            # Move the start index forward by `num`
            start += num
        # Join the parts with a space
        phrase = ' '.join(reshaped_string)

        all_possible_answers.append(phrase)
    all_possible_answers


    # Some of the stuff in this prompt can be done through algorithm.... Adding structural input (Stressing the FIRST, SECOND, THIRD). Also I like this chain of thought proces, need to conclude it and see how to trasnfer to another kind. (Finding definition and indicator, somewhat seperate to answer phrase, but leave possibility to it. Check is possible answers are meaningful before continuing. Check all possibilities, and compare the probability. )
    prompt = f"Give the cryptic crossword puzzle '{clue}', I know that this puzzle is a '{prompt_word_1}' type of cryptic crossword puzzles. The number in the clue hints that the answer is in shape {enumeration}, and the answer will be {number_of_letters} letters long. So if we take all the {prompt_word_1} letters of the the words in '{clue}' in order, we will form the string '{concat_tips}'. Then all the possible ansers are substrings of it (the consecutive substrings), so we get all possible answers: {all_possible_answers}. I want you to follow the following steps: For each string in the possible answers, \nFIRST, determine if it is a meaningful term. \nSECOND, if it's somewhat meaningful, What is the phrase in the original clue whose {prompt_word_1} letters formed this string. \nTHIRD, usually the phrase that forms answer will not intersect with the indicator phrase and definition phrase (sometimes there might be), so try to find what can be the indicator that indicate this puzzle is a '{prompt_word_1}' type of cryptic crossword puzzles, and what is the phrase that's left that might be the definition (Sometimes the definition might be the entire sentence, then this rule doesn't work). FOURTH, check if the definition we found suits the string's meaning. FINALLY, conclude that which string among all possible ansers are the actual answer."

    response = chat_bot.chat(prompt)
    # print(response)

    # extract information
    prompt_extract = f"Given the output:\n{response}, What is the answer? I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)
    return response_extract


# Alternate: Even/Odd

In [ ]:
def solve_even_odd_letters(sample, chat_bot, information_extractor):
    clue = sample["clue"]
    enumeration = sample["enumeration"]
    # get the number of letters of the answer (add all the numbers in enumeration)
    numbers_list = enumeration.strip('()').split(',')
    enumeration_numbers = [int(num) for num in numbers_list]
    number_of_letters = sum(enumeration_numbers)
    # get number of words (see how many numbers are there in enumeration)
    number_of_words_in_answer = enumeration.count(',') + 1

    # store the answer for comparison
    answer = sample['answer']

    # concatenate all the alternating sequences.
    clue_letters = re.sub(r"[^a-zA-Z]", '', clue).lower()
    even_letter_seq = clue_letters[::2]
    odd_letter_seq = clue_letters[1::2]

    # We will get all possibilities from both even and odd, and for the model to choose?
    all_possible_answers = []

    for letter_seq in [even_letter_seq, odd_letter_seq]:
        for i in range(len(letter_seq) - number_of_letters + 1):
            phrase = letter_seq[i:i+number_of_letters]

            # given a phrase of crrect length, make it the correct size (size of enumeration)
            reshaped_string = []
            start = 0
            for num in enumeration_numbers:
                # Extract the substring of length `num`
                word = phrase[start:start + num]
                reshaped_string.append(word)
                # Move the start index forward by `num`
                start += num
            # Join the parts with a space
            phrase = ' '.join(reshaped_string)

            all_possible_answers.append(phrase)
    all_possible_answers

    # start the prompt
    prompt = f"Give the cryptic crossword puzzle '{clue}', I know that this puzzle is a 'alternate letters' type of cryptic crossword puzzles. So if we take all the even letters of the clue, we will have '{even_letter_seq}', if we take all the odd letters of the clue, we will have '{odd_letter_seq}'. The number in the clue hints that the answer is in shape {enumeration}, and the answer will be {number_of_letters} letters long. Then all the possible ansers are substrings of it (the consecutive substrings), so we get all possible answers: {all_possible_answers}. I want you to follow the following steps: For each string in the possible answers, \nFIRST, determine if it is a meaningful term. \nSECOND, usually the phrase that forms answer will not intersect with the indicator phrase and definition phrase (sometimes there might be), so try to find what can be the indicator that indicate this puzzle is a 'alternate letters' type of cryptic crossword puzzles, and what is the phrase that's left that might be the definition (Sometimes the definition might be the entire sentence, then this rule doesn't work). THIRD, check if the definition we found suits the string's meaning. FINALLY, conclude that which string among all possible ansers are the actual answer."

    response = chat_bot.chat(prompt)
    # print(response)

    # extract information
    prompt_extract = f"Given the output:\n{response}, What is the answer? I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)
    return response_extract


# Homophones

# Letter Banks

In [ ]:
def solve_letter_bank(sample, chat_bot, information_extractor):

    clue = sample['clue']
    clue = re.sub(r"[^a-zA-Z']", ' ', clue)
    # get letter bank of clue
    clue_word_list = clue.lower().split()

    # get the letter bank for answer: From previous mapping, we already know answer only have english letters and space.
    answer = sample['answer'].lower()
    answer_word_list = answer.split()

    # get the answer length from enumeration field (I am lazy, just get it from answer field)
    enumeration = sample['enumeration']
    answer_length = len(answer)

    # First check if clue letter bank contains all letters of answers (Here we assume that is true: Later when writting classifiers we will think otherwise)

    # Now we find all possible isograms
    all_isograms = []
    for k in range(len(clue_word_list)-1):
        continue_checking_because_isogram_exists = False
        for i in range(len(clue_word_list) - k):
            # k_word_combo is ''.join(clue_word_list[i:i+k+1], letter bank of k_word_combo is set(k_word_combo)
            k_word_combo = ''.join(clue_word_list[i:i+k+1])
            k_word_combo_letter_bank = set(k_word_combo)

            # if the length of k_word_combo is bigger than answer length, than even if it's isogram, it's not the letter bank of the answer. Also, this isogram shouldn't be counted as isogram exist: If beside from this, all other k_word_combo are not isogram, then for larger k, this will not be a suitable isogram, and the rest of the others will not be isogram at all.
            if len(k_word_combo_letter_bank) > answer_length:
                continue

            # The we need to check if the k_word_combo is a isogram (It's a rule for letter bank)
            if len(k_word_combo_letter_bank) == len(k_word_combo):
                # if all the k_word_combo are not isogram, then for larger k, there will not be isogram.
                continue_checking_because_isogram_exists = True
                all_isograms.append(' '.join(clue_word_list[i:i+k+1]))

            else:
                continue

        if continue_checking_because_isogram_exists==False:
            break



    # Now we let LLM handle the rest
    prompt = f"Given the cryptic crossword puzzle {clue}, I know that it is a letter bank type crossword puzzle, which means there are an isogram in the clue (isogram is a word/phrase containing no repeated letters), and the answer are formed by by using each of these letters (but no others) at least once but repeating them as often as necessary. According to this rule, here are all possible isograms in this clue: {all_isograms}. And also, the hint number in the clue suggests that the answer is of shape {enumeration}. So now, I want you to solve the problem this way: \nFIRST, you will find the indicator phrase (the phrase that indicate that this puzzle is a letter bank type crossword puzzle), since letter bank puzzles resenbles to anagram puzzles, their indicators might also be similar; and then you will find the definition phrase (The phrase that gives definition to the answer). Of course you will not be sure, but you will look at what is the most likely phrase for each. \nSECOND, based on the indicator phrase and definition phrase, you can guess which isogram in the list of all possible isograms is the isogram that could potentially form the answer. (Usually the indicator phrase, the isogram and the definition phrase will not overlap, that means they will not share the same words in the clue). "

    response = chat_bot.chat(prompt)

    # extract information
    prompt_extract = f"Given the output:\n{response}, what are all the phrase to shuffle, the indicator, the definition and the isogram that the LLM chose? Give me in the form of tuple of three strings: (<indicator>, <definition>, <isogram>), I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)
    parsed_tuple = ast.literal_eval(response_extract)
    indicator, definition, isogram = parsed_tuple
    # print(parsed_tuple)

    sample# randomly suffle the letters
    list_of_letters = list(set(isogram))
    random.shuffle(list_of_letters)

    prompt = f"""Given the cryptic crossword puzzle {clue}, the indicator phrase '{indicator}' suggests that it is a letter bank type crossword puzzle, which means there are an isogram in the clue (isogram is a word/phrase containing no repeated letters), and the answer are formed by by using each of these letters (but no others) at least once but repeating them as often as necessary. And the phrase '{definition}' is likely the definition phrase that defines the answer. So based on the indicator phrase and definition phrase, we conclude that the isogram '{isogram}' is the isogram tha we use to construct the answer. And also, the hint number in the clue suggests that the answer is of shape {enumeration}. \n
    Now, I want you to solve the problem this way: \n
    Generate a word that:\n
    1. Matches the shape: {enumeration}\n
    2. Has the meaning: "{definition}"\n
    3. Uses every letters in {list_of_letters}, at least once (repetitions allowed).\n
    4. Does not use any letters outside this list. \n
    """
    response = chat_bot.chat(prompt)
    # print(response)

    # extract information
    prompt_extract = f"Given the output:\n{response}, What is the answer? I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)
    return response_extract


# Reversals

# Palindrome

In [ ]:
def solve_palindrom(sample, chat_bot, information_extractor):

    clue = sample['clue']
    # Remove the tuple at the end (assuming there are no other
    clue = re.sub(r"[^a-zA-Z']", ' ', clue).lower()

    # things about enumeration
    enumeration = sample["enumeration"]
    # get the number of letters of the answer (add all the numbers in enumeration)
    numbers_list = enumeration.strip('()').split(',')
    enumeration_numbers = [int(num) for num in numbers_list]
    number_of_letters = sum(enumeration_numbers)
    # get number of words (see how many numbers are there in enumeration)
    number_of_words_in_answer = enumeration.count(',') + 1

    # get the word length for answer
    answer = sample['answer'].lower().replace(" ", "")
    answer_len = len(answer)    # technically just add all the number in enumeratio together, i am lazy.



    prompt = f"Given the cryptic crossword puzzle {clue}, I know that it is a pallindrom type crossword puzzle. The hint number suggests that the shape of the answer is {enumeration}, so that means the phrase we want to reverse have exacty {number_of_letters} english letters, and also it is a pallindrom. You will solve this problem by the following steps: \nFirst, try to find the phrase that indicate that this puzzle is a pallindrom type crossword puzzle. \nSECOND, given the sentence, you will think of some pallindrom that has {number_of_letters} english letters, that might relate to this sentence semantically ---- As many as possible! \nTHIRD, Dobule check if the answers you gave are indeed pallindroms. FOURTH, check which one of the pallindroms suits the meaning of the sentence. FINALLY, conclude that which string among all possible ansers are the actual answer."

    response = chat_bot.chat(prompt)
    # print(response)

    # extract information
    prompt_extract = f"Given the output:\n{response}, What is the answer? I don't need other information."
    response_extract = information_extractor.chat(prompt_extract)
    return response_extract


# Load Chatbot

In [ ]:

def load_gpt_chat_bot(gpt_model = "gpt-4o-2024-08-06"):
    solver_system_prompt = "You are a helpful assistant. You are very good at solving cryptic crossword puzzles. "
    request_body = {
        "model": gpt_model,
        "messages": [{"role": "system", "content": solver_system_prompt}],
        "temperature": 0.7,
    }
    chat_bot = GPTChatBot(request_body)

    extractor_system_prompt = "You are served as a information extractor. You will be given the output of an LLM, and a question, and from the given output, you will extract the information that answers the question. Your output will be linked to a computer program, so you will be accurate and concise."

    # Load the 4o extractor instead
    request_body = {
        "model": "gpt-4o-2024-08-06",   # only the 4o model is good enough
        "messages": [{"role": "system", "content": extractor_system_prompt}],
        "temperature": 0.2,
    }
    information_extractor = GPTChatBot(request_body)
    return chat_bot, information_extractor


In [ ]:
def load_gemini_chat_bot(gemini_model = "gemini-1.5-pro"):
    solver_system_prompt = "You are a helpful assistant. You are very good at solving cryptic crossword puzzles. "

    chat_bot = GeminiChatBot(system_prompt=solver_system_prompt, gemini_model=gemini_model, temperature=0.7)

    extractor_system_prompt = "You are served as a information extractor. You will be given the output of an LLM, and a question, and from the given output, you will extract the information that answers the question. Your output will be linked to a computer program, so you will be accurate and concise."

    # information_extractor = GeminiChatBot(system_prompt=extractor_system_prompt, gemini_model=gemini_model, temperature=0.2)

    # Load the 4o extractor instead
    request_body = {
        "model": "gpt-4o-2024-08-06",   # only the 4o model is good enough
        "messages": [{"role": "system", "content": extractor_system_prompt}],
        "temperature": 0.2,
    }
    information_extractor = GPTChatBot(request_body)
    return chat_bot, information_extractor

# Test results
We will test for the success rate for this approach.

In [ ]:
dataset_hgggingface_dir = f'PromptEngineering/ProcessedDatasets/recognizable_data/'
datasets = load_from_disk(dataset_hgggingface_dir)

In [ ]:

from joblib import Memory
gemini_pro = "gemini-1.5-pro"
gemini_flash = "gemini-1.5-flash"
gpt_4o = "gpt-4o-2024-08-06"
gpt_4o_mini = "gpt-4o-mini"

# Work around for joblib caching in jupyter notebook "joblib persistence across sessions/machines"
def cache(mem, module, **mem_kwargs):
    # model is the notebook/python file name: Jupyter notebook's name is always changing so we need this work around
    def cache_(f):
        f.__module__ = module
        f.__qualname__ = f.__name__
        return mem.cache(f, **mem_kwargs)
    # return the cache function that will always create same name for cahce directory
    return cache_

# Create a memory object with a cache directory
memory = Memory(location="PromptEngineering/FunctionCache", verbose=0)

In [ ]:
preprocessing_added_columns = ['is_charade', 'is_double_definition', 'is_anagram', 'type', 'is_hidden_word', 'is_initial', 'is_final', 'is_even_letter', 'is_odd_letter', 'is_reverse', 'is_pallindrom', 'is_letter_bank']

In [ ]:
from tqdm import tqdm

def cache_test_all_models(solvables, solver_function, max_test_size=20):
    test_size = min(len(solvables), max_test_size)

    model_score = {'test_size': test_size, gemini_pro: 0, gemini_flash: 0, gpt_4o: 0, gpt_4o_mini: 0}
    for model in [gemini_flash, gemini_pro, gpt_4o, gpt_4o_mini]:
        for i in tqdm(range(test_size), ncols=100):
            sample = solvables[i]
            try:
                # cache the function
                response_extract = solver_function(sample, model, attempt=1)
                if response_extract.strip().lower() == sample['answer'].strip().lower():
                    model_score[model] += 1
            except:
                print(f"Failed to solve puzzle {i} with model {model}")
                continue
            # response_extract = solver_function(sample, model, attempt=1)
    return model_score


In [ ]:
# @title All Wrappers
# Carefullll Do not change this code!!!!

@cache(memory, "CoT")
def solve_anagram_wrapper(sample, model, attempt=1):
    if "gemini" in model:
        chat_bot, information_extractor = load_gemini_chat_bot(model)
    else:
        chat_bot, information_extractor = load_gpt_chat_bot(model)
    response_extract = solve_simple_anagram(sample, chat_bot, information_extractor)
    return response_extract


@cache(memory, "CoT")
def solve_charade_wrapper(sample, model, attempt=1):
    if "gemini" in model:
        chat_bot, information_extractor = load_gemini_chat_bot(model)
    else:
        chat_bot, information_extractor = load_gpt_chat_bot(model)
    response_extract = solve_charade(sample, chat_bot, information_extractor)
    return response_extract


@cache(memory, "CoT")
def solve_double_wrapper(sample, model, attempt=1):
    if "gemini" in model:
        chat_bot, information_extractor = load_gemini_chat_bot(model)
    else:
        chat_bot, information_extractor = load_gpt_chat_bot(model)
    response_extract = solve_double_definition(sample, chat_bot, information_extractor)
    return response_extract

@cache(memory, "CoT")
def solve_hidden_wrapper(sample, model, attempt=1):
    if "gemini" in model:
        chat_bot, information_extractor = load_gemini_chat_bot(model)
    else:
        chat_bot, information_extractor = load_gpt_chat_bot(model)
    response_extract = solve_simple_hidden_word(sample, chat_bot, information_extractor)
    return response_extract


@cache(memory, "CoT")
def solve_tip_wrapper(sample, model, attempt=1):
    if "gemini" in model:
        chat_bot, information_extractor = load_gemini_chat_bot(model)
    else:
        chat_bot, information_extractor = load_gpt_chat_bot(model)
    response_extract = solve_initials_finals(sample, chat_bot, information_extractor)
    return response_extract


@cache(memory, "CoT")
def solve_alternate_wrapper(sample, model, attempt=1):
    if "gemini" in model:
        chat_bot, information_extractor = load_gemini_chat_bot(model)
    else:
        chat_bot, information_extractor = load_gpt_chat_bot(model)
    response_extract = solve_even_odd_letters(sample, chat_bot, information_extractor)
    return response_extract


@cache(memory, "CoT")
def solve_letter_bank_wrapper(sample, model, attempt=1):
    if "gemini" in model:
        chat_bot, information_extractor = load_gemini_chat_bot(model)
    else:
        chat_bot, information_extractor = load_gpt_chat_bot(model)
    response_extract = solve_letter_bank(sample, chat_bot, information_extractor)
    return response_extract

@cache(memory, "CoT")
def solve_palindrome_wrapper(sample, model, attempt=1):
    if "gemini" in model:
        chat_bot, information_extractor = load_gemini_chat_bot(model)
    else:
        chat_bot, information_extractor = load_gpt_chat_bot(model)
    response_extract = solve_palindrom(sample, chat_bot, information_extractor)
    return response_extract


In [ ]:
# @title Test Anagram
solvables = datasets.filter(lambda sample: sample['is_anagram'] == True)
solvables = solvables.remove_columns(preprocessing_added_columns)

cache_test_all_models(solvables['test'], solver_function=solve_anagram_wrapper)

100%|███████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 88.89it/s]


{'test_size': 20,
 'gemini-1.5-pro': 5,
 'gemini-1.5-flash': 1,
 'gpt-4o-2024-08-06': 11,
 'gpt-4o-mini': 3}

In [ ]:
# @title Test charade
solvables = datasets.filter(lambda sample: sample['is_charade'] == True)
solvables = solvables.remove_columns(preprocessing_added_columns)['test']
cache_test_all_models(solvables, solver_function=solve_charade_wrapper)

100%|██████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 108.73it/s]


{'test_size': 20,
 'gemini-1.5-pro': 13,
 'gemini-1.5-flash': 8,
 'gpt-4o-2024-08-06': 12,
 'gpt-4o-mini': 4}

In [ ]:
# @title Test Double definition
solvables = datasets.filter(lambda sample: sample['is_double_definition'] == True)
solvables = solvables.remove_columns(preprocessing_added_columns)['test']
cache_test_all_models(solvables, solver_function=solve_double_wrapper)

Filter:   0%|          | 0/470852 [00:00<?, ? examples/s]

Filter:   0%|          | 0/26204 [00:00<?, ? examples/s]

Filter:   0%|          | 0/26205 [00:00<?, ? examples/s]

100%|███████████████████████████████████████████████████████████████| 13/13 [03:04<00:00, 14.17s/it]


{'test_size': 13,
 'gemini-1.5-pro': 4,
 'gemini-1.5-flash': 3,
 'gpt-4o-2024-08-06': 5,
 'gpt-4o-mini': 3}

In [ ]:
# @title Test Hidden Words
solvables = datasets.filter(lambda sample: sample['is_hidden_word'] == True)
solvables = solvables.remove_columns(preprocessing_added_columns)['test']
cache_test_all_models(solvables, solver_function=solve_hidden_wrapper)


100%|██████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 109.19it/s]


{'test_size': 20,
 'gemini-1.5-pro': 5,
 'gemini-1.5-flash': 6,
 'gpt-4o-2024-08-06': 15,
 'gpt-4o-mini': 11}

In [ ]:
# @title Test initial/finals
solvables = datasets.filter(lambda sample: (sample['is_initial'] == True) or (sample['is_final'] == True))
solvables = solvables['test']
cache_test_all_models(solvables, solver_function=solve_tip_wrapper)


100%|███████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 98.12it/s]


{'test_size': 20,
 'gemini-1.5-pro': 18,
 'gemini-1.5-flash': 19,
 'gpt-4o-2024-08-06': 20,
 'gpt-4o-mini': 20}

In [ ]:
# @title Test Alternate
solvables = datasets.filter(lambda sample: (sample['is_even_letter'] == True) or (sample['is_odd_letter'] == True))
solvables = solvables.remove_columns(preprocessing_added_columns)['test']
cache_test_all_models(solvables, solver_function=solve_alternate_wrapper)

Filter:   0%|          | 0/470852 [00:00<?, ? examples/s]

Filter:   0%|          | 0/26204 [00:00<?, ? examples/s]

Filter:   0%|          | 0/26205 [00:00<?, ? examples/s]

100%|███████████████████████████████████████████████████████████████| 20/20 [02:29<00:00,  7.48s/it]


{'test_size': 20,
 'gemini-1.5-pro': 14,
 'gemini-1.5-flash': 18,
 'gpt-4o-2024-08-06': 15,
 'gpt-4o-mini': 18}

In [ ]:
# @title Test Palindrom
solvables = datasets.filter(lambda sample: sample['is_pallindrom'] == True)
solvables = solvables.remove_columns(preprocessing_added_columns)['test']
cache_test_all_models(solvables, solver_function=solve_palindrome_wrapper)

Filter:   0%|          | 0/470852 [00:00<?, ? examples/s]

Filter:   0%|          | 0/26204 [00:00<?, ? examples/s]

Filter:   0%|          | 0/26205 [00:00<?, ? examples/s]

100%|███████████████████████████████████████████████████████████████| 20/20 [01:25<00:00,  4.30s/it]


{'test_size': 20,
 'gemini-1.5-pro': 11,
 'gemini-1.5-flash': 9,
 'gpt-4o-2024-08-06': 15,
 'gpt-4o-mini': 2}